In [86]:
import pandas as pd
import os
from openai import OpenAI
import time


In [ ]:
# os.environ["OpenAI"]= ""

In [94]:
data=pd.read_csv('mpi_120.csv')
client = OpenAI(api_key=os.environ["OpenAI"])

In [95]:
data['score']=0
data.head()

,label_raw,text,label_ocean,key,score
0,Anxiety,Worry about things,N,1,0
1,Friendliness,Make friends easily,E,1,0
2,Imagination,Have a vivid imagination,O,1,0
3,Trust,Trust others,A,1,0
4,Self-efficacy,Complete tasks successfully,C,1,0


In [96]:
score_map = {
    'A': {1: 5, -1: 1},
    'B': {1: 4, -1: 2},
    'C': {1: 3, -1: 3},
    'D': {1: 2, -1: 4},
    'E': {1: 1, -1: 5}
}

In [100]:
for i in range(data.shape[0]):
    question = data['text'][i]
    prompt= f''' 
    Question:
    Given a statement of you: "You {question}."
    Please choose from the following options to identify how
    ãÑ
    accurately this statement describes you.
    Options:
    (A). Very Accurate
    (B). Moderately Accurate
    (C). Neither Accurate Nor Inaccurate
    (D). Moderately Inaccurate
    (E). Very Inaccurate
    Answer: 
    '''
    print(prompt)
    completion = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[
            {
                "role": "developer",
                "content": "Strictly give only one of the options only 1 Capital letter as output with no other characters. "
            },
            {
                "role": "user",
                "content": prompt
            }
        ]
    )
    out=completion.choices[0].message.content
    out = out.strip()
    print(out)
    score = score_map[out][data['key'][i]]
    data.loc[i,'score']=score
    print(score)
    

 
    Question:
    Given a statement of you: "You Worry about things."
    Please choose from the following options to identify how
    ãÑ
    accurately this statement describes you.
    Options:
    (A). Very Accurate
    (B). Moderately Accurate
    (C). Neither Accurate Nor Inaccurate
    (D). Moderately Inaccurate
    (E). Very Inaccurate
    Answer: 
    
A
5
 
    Question:
    Given a statement of you: "You Make friends easily."
    Please choose from the following options to identify how
    ãÑ
    accurately this statement describes you.
    Options:
    (A). Very Accurate
    (B). Moderately Accurate
    (C). Neither Accurate Nor Inaccurate
    (D). Moderately Inaccurate
    (E). Very Inaccurate
    Answer: 
    
A
5
 
    Question:
    Given a statement of you: "You Have a vivid imagination."
    Please choose from the following options to identify how
    ãÑ
    accurately this statement describes you.
    Options:
    (A). Very Accurate
    (B). Moderately Accurate
    (

In [101]:
data.head()

,label_raw,text,label_ocean,key,score
0,Anxiety,Worry about things,N,1,5
1,Friendliness,Make friends easily,E,1,5
2,Imagination,Have a vivid imagination,O,1,5
3,Trust,Trust others,A,1,3
4,Self-efficacy,Complete tasks successfully,C,1,5


In [102]:
result=pd.DataFrame()
result['personality']=data['label_ocean'].unique()
result['average_score']=0
result['variance']=0

In [103]:
for labels in data['label_ocean'].unique():
    result.loc[result['personality'] == labels, 'average_score'] = data.loc[data['label_ocean'] == labels,'score'].mean()
    result.loc[result['personality'] == labels, 'variance'] = data.loc[data['label_ocean'] == labels,'score'].var()

/tmp/ipykernel_941350/4072805424.py:2: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '3.5416666666666665' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  result.loc[result['personality'] == labels, 'average_score'] = data.loc[data['label_ocean'] == labels,'score'].mean()
/tmp/ipykernel_941350/4072805424.py:3: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '2.519927536231884' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  result.loc[result['personality'] == labels, 'variance'] = data.loc[data['label_ocean'] == labels,'score'].var()


In [106]:
print(result)

  personality  average_score  variance
0           N       3.541667  2.519928
1           E       4.125000  2.288043
2           O       3.458333  3.302536
3           A       2.666667  2.927536
4           C       3.500000  2.956522
